In [ ]:
"""
Tokenizer

CS 6200 Information Retreival 
Homework 2
Melanie Platt

Implementation of a document indexer. Parse, tokenize, and index documents to use ranking models on for query searching.
                                                                    
"""

In [6]:
import re
from nltk.stem import PorterStemmer 
import pickle
import os
import time
from collections import OrderedDict 

STOP_PATH = "C:/6200-IR/homework-2-mplatt27/stop_words_list.txt"
STEMMING = False
if STEMMING:
    ps = PorterStemmer() 

In [8]:
""" Part 1: Tokenizing """

# 1. For each document, create a doc hash and save in a seperate file that maps doc id to doc hash (e.g, AP89000 --> 1)
# 2. Create a dictionary that maps each doc hash to a list of tuples. Tuples will represent tokens for that document 
#    (term, position-in-doc)


"""
Tokenizer creates tokens from each document to later be indexed. It can save the doc hashes to a txt file.
It can also save the tokens to a pickle to later be used. 
"""
class Tokenizer:
    
    def __init__(self):
        self.doc_dict = OrderedDict()
        self.doc_hashes = {}
        
        self.stop_words = []
        for line in open(STOP_PATH, encoding="ISO-8859-1", errors='ignore'):
            self.stop_words.append(line.strip())
        print("Stop words list created!")
        
        print("Indexer initialized. Begining to parse documents...")
        
        
    """
    Function: get_files_in_dir()
    Input: folder_path: a path to a folder of files to be indexed
    Returns: file_path_list: a list of paths to each file in the folder
    Does: Gets the names of all files in the folder then appends each
    file's path name to a list to return
    """ 
    def get_files_in_dir(self, folder_path):
        # gets all names of files in directory
        file_list = os.listdir(folder_path)

        # append them to list with their full paths
        file_path_list = []
        for file in file_list:
            file_path_list.append(os.path.join(folder_path, file))

        return file_path_list


    """
    Function: get_data_from_text_file()
    Input: file: a single file that may contain multiple documents to be indexed
    Returns: data: a list of lists; each sub-list is a line from the file
    Does: reads each line of the file and appends it in a list to data
    """
    def get_data_from_text_file(self, file):
        # declare an empty list for the data
        data = []
        for line in open(file, encoding="ISO-8859-1", errors='ignore'):
            data.append(line.strip())
        return data
    
    
    """
    Function: tokenize_text
    Input: A string of text from the document to be tokenized
    Output: A list of tokens as tuples (term, position in doc)
    """
    def tokenize_text(self, text):
        
        tokens = []
        text_list= re.findall(r"\w+|[^\w\s]|[\n\r]+", text, re.UNICODE) # split on whitespace/punctuation/newline
        
        for pos, token in enumerate(text_list):
            if token.isalnum() and token not in self.stop_words: # leave out punct and stop words

                # stem the token if required
                if STEMMING:
                    token = ps.stem(token)

                # create a tuple (term, position) and place in list to later be added to doc_dict
                # note: position refers to full term in the doc (not char index in doc as string)
                token_tuple = (token, pos)
                tokens.append(token_tuple)
                
        return tokens
        


    """
    Function: parse_docs()
    Input: files: a list of each file path that we want to index (each file may contain multiple docs)
    Returns: doc_dict (a dictionary of documents mapped to tuple tokens)
    Does: For each file, get the filename and the data (as a list of lists, each sub-list is a line from the file).
    Then separate out each document per file. For each doc, create hash and write to file. Extract tokens; for each token
    place in token dict and write to file at end. Place all tokens for each doc in doc_dict to return. 
    """
    def parse_docs(self, files):

        # iterate over each file (each file may contain multiple documents)
        doc_hash = 1
        for count, _file in enumerate(files):
            file_name = _file[_file.rfind("/") + 1:]
            if file_name == 'readme':
                print("found readme")
            if file_name != 'readme':
                data = self.get_data_from_text_file(_file)  # everything in one file (may have multiple documents)

                # iterate over each line and add it to docs, which will be each doc as its own list (tokenized)
                docs = []
                current = []
                for element in data:
                    doc_end_tag = "</DOC>"
                    if doc_end_tag not in element:
                        current.append(element)
                    else:
                        current.append(element)
                        docs.append(current)
                        current = []

                # docs is a list of lists (one doc for each sub-list)
                # for each doc we now need to get the docno (id), and text (for tokenization)
                for doc in docs:
                    doc = " ".join(doc)
                    docend = doc.find("</DOC>")  # index where document end tag starts
                    doc_sub = doc[:docend]  # splices off doc end tag
                    docno_s = doc_sub.find("<DOCNO>") + len("<DOCNO>")  # index after the doc no start tag
                    docno_e = doc_sub.find("</DOCNO>")  # index at start of doc no tag
                    docno = doc_sub[docno_s:docno_e].strip()  # this will be the _id
                    text_s = doc_sub.find("<TEXT>") + len("<TEXT>")  # index at end of text start tag
                    text_e = doc_sub.find("</TEXT")  # start index of text end tag
                    text = doc_sub[text_s:text_e].strip() + "\\ln"  # splice out text
                    text = text.lower()

                    # collect list of tokens and add to doc_dict
                    tokens = self.tokenize_text(text)
                    self.doc_dict[doc_hash] = tokens
                    
                    # add doc id to hash map along with tokens (the len(d) for this doc; will need later)
                    self.doc_hashes[docno] = (doc_hash, len(tokens))

                    doc_hash += 1
                                              
                                              
    """
    Function: write_doc_hashes_to_file
    Input: path for where to save the file
    Returns: None
    Does: Writes the doc hashes dictionary to a file to be used later. 
    """
    def write_doc_hashes_to_file(self, save_path):
        outF = open(save_path, "w")
        for docid, doc_tuple in self.doc_hashes.items():
            outF.write(docid + " " + str(doc_tuple[0]) + "|" + str(doc_tuple[1]))
            outF.write("\n")
        outF.close()
        print("File saved!")
        
        
    """
    Function: write_doc_dict_to_pickle
    Input: path for where to save the file
    Returns: None
    Does: Writes the doc dict of term tuples to a pickle to be used later. 
    """
    def write_doc_dict_to_pickle(self, save_path):
        save_path = save_path + "doc_dict_nostemming.pickle"
        with open(save_path, 'wb') as handle:
            pickle.dump(self.doc_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Pickle saved!") 


In [3]:
""" Main code """

start_time = time.time()

# path to documents
file_path = "C:/6200-IR/homework1-mplatt27/IR_data/AP_DATA/ap89_collection/"

# create tokenizer with stemming
tokenizer = Tokenizer()
all_files_paths = tokenizer.get_files_in_dir(file_path)
tokenizer.parse_docs(all_files_paths)
print("Text has been parsed!")
print("--- %s seconds ---" % (time.time() - start_time))


Stop words list created!
Indexer initialized. Begining to parse documents...
found readme
Text has been parsed!
--- 478.50155115127563 seconds ---


In [4]:
# save to file (this version has stemming)
tokenizer.write_doc_hashes_to_file("C:/6200-IR/doc_hashes.txt")
tokenizer.write_doc_dict_to_pickle("C:/6200-IR/")

AP890101-0001 1 571
AP890101-0002 2 443
AP890101-0003 3 130
AP890101-0004 4 358
AP890101-0005 5 36
AP890101-0006 6 476
AP890101-0007 7 186
AP890101-0008 8 278
AP890101-0009 9 371
AP890101-0010 10 146
AP890101-0011 11 217
AP890101-0012 12 282
AP890101-0013 13 375
AP890101-0014 14 433
AP890101-0015 15 265
AP890101-0016 16 127
AP890101-0017 17 431
AP890101-0018 18 281
AP890101-0019 19 118
AP890101-0020 20 134
AP890101-0021 21 296
AP890101-0022 22 177
AP890101-0023 23 282
AP890101-0024 24 269
AP890101-0025 25 276
AP890101-0026 26 183
AP890101-0027 27 113
AP890101-0028 28 217
AP890101-0029 29 165
AP890101-0030 30 533
AP890101-0031 31 253
AP890101-0032 32 55
AP890101-0033 33 43
AP890101-0034 34 208
AP890101-0035 35 155
AP890101-0036 36 262
AP890101-0037 37 234
AP890101-0038 38 250
AP890101-0039 39 118
AP890101-0040 40 281
AP890101-0041 41 355
AP890101-0042 42 149
AP890101-0043 43 113
AP890101-0044 44 114
AP890101-0045 45 222
AP890101-0046 46 120
AP890101-0047 47 425
AP890101-0048 48 112
AP89

AP890104-0152 632 377
AP890104-0153 633 135
AP890104-0154 634 320
AP890104-0155 635 104
AP890104-0156 636 270
AP890104-0157 637 217
AP890104-0158 638 410
AP890104-0159 639 78
AP890104-0160 640 283
AP890104-0161 641 147
AP890104-0162 642 14
AP890104-0163 643 498
AP890104-0164 644 207
AP890104-0165 645 286
AP890104-0166 646 183
AP890104-0167 647 202
AP890104-0168 648 392
AP890104-0169 649 211
AP890104-0170 650 495
AP890104-0171 651 498
AP890104-0172 652 214
AP890104-0173 653 282
AP890104-0174 654 227
AP890104-0175 655 118
AP890104-0176 656 361
AP890104-0177 657 17
AP890104-0178 658 427
AP890104-0179 659 499
AP890104-0180 660 384
AP890104-0181 661 110
AP890104-0182 662 283
AP890104-0183 663 331
AP890104-0184 664 271
AP890104-0185 665 201
AP890104-0186 666 167
AP890104-0187 667 144
AP890104-0188 668 138
AP890104-0189 669 232
AP890104-0190 670 430
AP890104-0191 671 293
AP890104-0192 672 115
AP890104-0193 673 339
AP890104-0194 674 82
AP890104-0195 675 359
AP890104-0196 676 225
AP890104-0197 

AP890106-0208 1256 130
AP890106-0209 1257 497
AP890106-0210 1258 39
AP890106-0211 1259 56
AP890106-0212 1260 502
AP890106-0213 1261 134
AP890106-0214 1262 237
AP890106-0215 1263 341
AP890106-0216 1264 277
AP890106-0217 1265 80
AP890106-0218 1266 360
AP890106-0219 1267 199
AP890106-0220 1268 271
AP890106-0221 1269 213
AP890106-0222 1270 165
AP890106-0223 1271 170
AP890106-0224 1272 225
AP890106-0225 1273 271
AP890106-0226 1274 260
AP890106-0227 1275 189
AP890106-0228 1276 179
AP890106-0229 1277 210
AP890106-0230 1278 12
AP890106-0231 1279 187
AP890106-0232 1280 141
AP890106-0233 1281 26
AP890106-0234 1282 88
AP890106-0235 1283 270
AP890106-0236 1284 54
AP890106-0237 1285 465
AP890106-0238 1286 224
AP890106-0239 1287 114
AP890106-0240 1288 177
AP890106-0241 1289 440
AP890106-0242 1290 210
AP890106-0243 1291 571
AP890106-0244 1292 265
AP890106-0245 1293 448
AP890106-0246 1294 354
AP890106-0247 1295 404
AP890106-0248 1296 265
AP890106-0249 1297 144
AP890106-0250 1298 14
AP890106-0251 1299 

AP890109-0341 1905 272
AP890109-0342 1906 256
AP890109-0343 1907 118
AP890109-0344 1908 188
AP890109-0345 1909 227
AP890109-0346 1910 385
AP890109-0347 1911 764
AP890109-0348 1912 328
AP890109-0349 1913 92
AP890110-0001 1914 255
AP890110-0002 1915 268
AP890110-0003 1916 134
AP890110-0004 1917 29
AP890110-0005 1918 403
AP890110-0006 1919 141
AP890110-0007 1920 400
AP890110-0008 1921 97
AP890110-0009 1922 162
AP890110-0010 1923 294
AP890110-0011 1924 70
AP890110-0012 1925 257
AP890110-0013 1926 95
AP890110-0014 1927 330
AP890110-0015 1928 432
AP890110-0016 1929 346
AP890110-0017 1930 419
AP890110-0018 1931 156
AP890110-0019 1932 320
AP890110-0020 1933 474
AP890110-0021 1934 305
AP890110-0022 1935 175
AP890110-0023 1936 407
AP890110-0024 1937 314
AP890110-0025 1938 220
AP890110-0026 1939 220
AP890110-0027 1940 28
AP890110-0028 1941 16
AP890110-0029 1942 431
AP890110-0030 1943 329
AP890110-0031 1944 391
AP890110-0032 1945 126
AP890110-0033 1946 324
AP890110-0034 1947 330
AP890110-0035 1948

AP890112-0048 2557 76
AP890112-0049 2558 112
AP890112-0050 2559 269
AP890112-0051 2560 221
AP890112-0052 2561 213
AP890112-0053 2562 339
AP890112-0054 2563 185
AP890112-0055 2564 282
AP890112-0056 2565 243
AP890112-0057 2566 325
AP890112-0058 2567 31
AP890112-0059 2568 161
AP890112-0060 2569 317
AP890112-0061 2570 326
AP890112-0062 2571 347
AP890112-0063 2572 169
AP890112-0064 2573 221
AP890112-0065 2574 250
AP890112-0066 2575 254
AP890112-0067 2576 234
AP890112-0068 2577 172
AP890112-0069 2578 231
AP890112-0070 2579 243
AP890112-0071 2580 194
AP890112-0072 2581 380
AP890112-0073 2582 189
AP890112-0074 2583 400
AP890112-0075 2584 152
AP890112-0076 2585 93
AP890112-0077 2586 181
AP890112-0078 2587 202
AP890112-0079 2588 328
AP890112-0080 2589 168
AP890112-0081 2590 173
AP890112-0082 2591 19
AP890112-0083 2592 301
AP890112-0084 2593 149
AP890112-0085 2594 123
AP890112-0086 2595 224
AP890112-0087 2596 117
AP890112-0088 2597 187
AP890112-0089 2598 102
AP890112-0090 2599 222
AP890112-0091 2

...

AP891217-0012 81818 156
AP891217-0013 81819 392
AP891217-0014 81820 338
AP891217-0015 81821 551
AP891217-0016 81822 160
AP891217-0017 81823 179
AP891217-0018 81824 327
AP891217-0019 81825 145
AP891217-0020 81826 58
AP891217-0021 81827 376
AP891217-0022 81828 417
AP891217-0023 81829 428
AP891217-0024 81830 259
AP891217-0025 81831 193
AP891217-0026 81832 292
AP891217-0027 81833 258
AP891217-0028 81834 365
AP891217-0029 81835 76
AP891217-0030 81836 152
AP891217-0031 81837 247
AP891217-0032 81838 151
AP891217-0033 81839 101
AP891217-0034 81840 189
AP891217-0035 81841 60
AP891217-0036 81842 408
AP891217-0037 81843 202
AP891217-0038 81844 198
AP891217-0039 81845 246
AP891217-0040 81846 357
AP891217-0041 81847 197
AP891217-0042 81848 96
AP891217-0043 81849 258
AP891217-0044 81850 332
AP891217-0045 81851 331
AP891217-0046 81852 257
AP891217-0047 81853 476
AP891217-0048 81854 161
AP891217-0049 81855 257
AP891217-0050 81856 150
AP891217-0051 81857 381
AP891217-0052 81858 243
AP891217-0053 81859 

AP891220-0034 82466 362
AP891220-0035 82467 23
AP891220-0036 82468 447
AP891220-0037 82469 495
AP891220-0038 82470 9
AP891220-0039 82471 102
AP891220-0040 82472 221
AP891220-0041 82473 183
AP891220-0042 82474 227
AP891220-0043 82475 158
AP891220-0044 82476 222
AP891220-0045 82477 183
AP891220-0046 82478 292
AP891220-0047 82479 302
AP891220-0048 82480 294
AP891220-0049 82481 161
AP891220-0050 82482 221
AP891220-0051 82483 13
AP891220-0052 82484 706
AP891220-0053 82485 107
AP891220-0054 82486 363
AP891220-0055 82487 11
AP891220-0056 82488 229
AP891220-0057 82489 297
AP891220-0058 82490 498
AP891220-0059 82491 340
AP891220-0060 82492 116
AP891220-0061 82493 133
AP891220-0062 82494 144
AP891220-0063 82495 185
AP891220-0064 82496 227
AP891220-0065 82497 312
AP891220-0066 82498 84
AP891220-0067 82499 62
AP891220-0068 82500 352
AP891220-0069 82501 328
AP891220-0070 82502 467
AP891220-0071 82503 19
AP891220-0072 82504 4
AP891220-0073 82505 258
AP891220-0074 82506 210
AP891220-0075 82507 332
AP

AP891222-0119 83131 119
AP891222-0120 83132 258
AP891222-0121 83133 102
AP891222-0122 83134 274
AP891222-0123 83135 133
AP891222-0124 83136 268
AP891222-0125 83137 97
AP891222-0126 83138 273
AP891222-0127 83139 186
AP891222-0128 83140 364
AP891222-0129 83141 284
AP891222-0130 83142 362
AP891222-0131 83143 445
AP891222-0132 83144 531
AP891222-0133 83145 407
AP891222-0134 83146 20
AP891222-0135 83147 297
AP891222-0136 83148 414
AP891222-0137 83149 215
AP891222-0138 83150 197
AP891222-0139 83151 699
AP891222-0140 83152 263
AP891222-0141 83153 361
AP891222-0142 83154 34
AP891222-0143 83155 316
AP891222-0144 83156 338
AP891222-0145 83157 150
AP891222-0146 83158 278
AP891222-0147 83159 192
AP891222-0148 83160 686
AP891222-0149 83161 54
AP891222-0150 83162 318
AP891222-0151 83163 293
AP891222-0152 83164 249
AP891222-0153 83165 152
AP891222-0154 83166 107
AP891222-0155 83167 363
AP891222-0156 83168 278
AP891222-0157 83169 401
AP891222-0158 83170 332
AP891222-0159 83171 514
AP891222-0160 83172 

AP891226-0165 83793 81
AP891226-0166 83794 175
AP891226-0167 83795 16
AP891226-0168 83796 371
AP891226-0169 83797 549
AP891226-0170 83798 394
AP891226-0171 83799 362
AP891226-0172 83800 369
AP891226-0173 83801 681
AP891226-0174 83802 208
AP891226-0175 83803 418
AP891226-0176 83804 187
AP891226-0177 83805 23
AP891226-0178 83806 54
AP891226-0179 83807 319
AP891226-0180 83808 257
AP891226-0181 83809 20
AP891226-0182 83810 10
AP891226-0183 83811 73
AP891226-0184 83812 251
AP891226-0185 83813 354
AP891226-0186 83814 163
AP891226-0187 83815 184
AP891226-0188 83816 85
AP891226-0189 83817 133
AP891226-0190 83818 172
AP891226-0191 83819 190
AP891226-0192 83820 281
AP891226-0193 83821 189
AP891226-0194 83822 206
AP891226-0195 83823 137
AP891226-0196 83824 208
AP891226-0197 83825 26
AP891226-0198 83826 114
AP891226-0199 83827 116
AP891226-0200 83828 67
AP891226-0201 83829 166
AP891226-0202 83830 228
AP891226-0203 83831 472
AP891226-0204 83832 519
AP891226-0205 83833 384
AP891226-0206 83834 107
AP

AP891229-0181 84445 32
AP891229-0182 84446 5
AP891229-0183 84447 570
AP891229-0184 84448 245
AP891229-0185 84449 333
AP891229-0186 84450 323
AP891229-0187 84451 96
AP891229-0188 84452 177
AP891229-0189 84453 139
AP891229-0190 84454 47
AP891229-0191 84455 179
AP891229-0192 84456 20
AP891229-0193 84457 166
AP891229-0194 84458 334
AP891229-0195 84459 122
AP891229-0196 84460 419
AP891229-0197 84461 12
AP891229-0198 84462 165
AP891229-0199 84463 139
AP891229-0200 84464 121
AP891229-0201 84465 240
AP891229-0202 84466 121
AP891229-0203 84467 293
AP891229-0204 84468 164
AP891229-0205 84469 156
AP891229-0206 84470 583
AP891229-0207 84471 362
AP891229-0208 84472 23
AP891229-0209 84473 290
AP891229-0210 84474 326
AP891229-0211 84475 63
AP891229-0212 84476 185
AP891229-0213 84477 461
AP891229-0214 84478 227
AP891229-0215 84479 73
AP891229-0216 84480 225
AP891229-0217 84481 106
AP891229-0218 84482 336
AP891229-0219 84483 481
AP891229-0220 84484 167
AP891229-0221 84485 74
AP891229-0222 84486 84
AP89

In [9]:
# create tokenizer with stemming
start_time = time.time()
print("New creating tokens without stemming") # change variable at the top
tokenizer = Tokenizer()
all_files_paths = tokenizer.get_files_in_dir(file_path)
tokenizer.parse_docs(all_files_paths)
print("Text has been parsed!")
print("--- %s seconds ---" % (time.time() - start_time))

# write pickle for file without stemming, doc hashes file will be the same
tokenizer.write_doc_hashes_to_file("C:/6200-IR/doc_hashes_nostemming.txt")
tokenizer.write_doc_dict_to_pickle("C:/6200-IR/")

New creating tokens without stemming
Stop words list created!
Indexer initialized. Begining to parse documents...
found readme
Text has been parsed!
--- 145.04304480552673 seconds ---
File saved!
Pickle saved!
